In [1]:
from framework.simulation import FaradayThinSource,FaradayThickSource
from framework.reconstruction import Parameter
import numpy as np
from IPython.display import clear_output
from framework.io import Reader, Writer
from framework.base import Dataset
from framework.transformers import DFT1D, NUFFT1D
import matplotlib.pyplot as plt
from framework.objectivefunction import OFunction
from framework.utils import real_to_complex, complex_to_real, find_pixel
from framework.objectivefunction import TSV, TV, L1, Chi2
from framework.optimization import FISTA, ADMM, SDMM, GradientBasedMethod
from framework.utils import Gaussian
from framework.dictionaries.discrete import DiscreteWavelet
from framework.dictionaries.undecimated import UndecimatedWavelet
from framework.transformers import Gridding
from scipy.constants import c
from pynufft import NUFFT
from scipy import signal as sci_signal
import itertools
import copy
import pandas as pd
import time
import pywt
import os
%matplotlib inline
#np.random.seed(666)

In [2]:
def gini_coefficient(w):
    # Order vector
    w_ordered = np.sort(np.abs(w), kind="stable")
    l1_norm = np.sum(np.abs(w_ordered))
    M = len(w_ordered)
    m = np.arange(0, M)
    const = (M - m + 1.5)/M
    if l1_norm == 0.0:
        coeff = np.nan
    else:
        coeff = np.sum((w_ordered/l1_norm)*const)
    return 1.0 - 2.0 * coeff

In [3]:
def chi2_calc(residuals):
    if residuals.dtype == np.complex64 or residuals.dtype == np.complex128:
        data = residuals.real**2 + residuals.imag**2
    else:
        data = residuals**2
    return np.sum(data)

In [4]:
def aicbic(residuals, x):
    rss = chi2_calc(residuals)
    if x.dtype == np.complex64 or x.dtype == np.complex128:
        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
    else:     
        df = np.count_nonzero(x)
    l = 2*len(residuals)
    return l*np.log(rss/l) + 2*df, l*np.log(rss/l) + df*np.log(l)

In [5]:
def list_to2darray(x: list = None, cols: int = None, dtype=None):
    b = list(map(list, zip(*[iter(x)]*cols)))
    #b = [x[cols*i : cols*(i+1)] for i in range(rows)]
    if dtype is None:
        return np.array(b)
    else:
        return np.array(b, dtype=dtype)
    

In [6]:
class statistics:
    def __init__(self, m, n, z):
        self.sum = np.zeros((m,n), dtype=np.float32)
        self.sum2 = np.zeros((m,n), dtype=np.float32)
        self.n = z * np.ones((m,n), dtype=np.int32)
    
    def cumul(self, x):
        x_values = np.where(x != np.nan, x, 0.0)
        subtract = np.where(x == np.nan, -1, 0)
        self.sum  += x_values
        self.sum2 += x_values * x_values
        self.n += subtract
    
    def mean(self):
        return np.where(self.n > 0, self.sum / self.n, np.nan)
    
    def std(self):
        return np.where(self.n > 0, np.sqrt(self.sum2/self.n - self.sum*self.sum/self.n/self.n), np.nan)

In [7]:
class Test:
    def __init__(self, nu_min=None, nu_max=None, nchannels=None, noise_frac=None, remove_frac=None, use_gridding=False, ftransform="nufft", use_wavelet=None, source_1=None, source_2=None, scenario=1):
        self.nu_min = nu_min
        self.nu_max = nu_max
        self.nchannels = nchannels
        self.noise_frac = noise_frac
        self.remove_frac = remove_frac
        self.use_gridding=use_gridding
        self.use_wavelet = use_wavelet
        self.ftransform = ftransform
        self.scenario = scenario
        self.nu = np.linspace(start=nu_min, stop=nu_max, num=nchannels)
        self.source_1 = copy.deepcopy(source_1)
        self.source_2 = copy.deepcopy(source_2)

        if self.source_1 is not None:
            self.source_1.nu = self.nu
            self.source_1.simulate()
                
        if self.source_2 is not None:
            self.source_2.nu = self.nu
            self.source_2.simulate()
                
        if scenario == 1:
            self.source = self.source_1
        elif scenario == 2:
            self.source = self.source_2
        elif scenario == 3:
            self.source = self.source_1 + self.source_2
        else:
            raise ValueError("This scenario does not exist")
            
        if remove_frac:
            self.source.remove_channels(remove_frac, np.random.RandomState(int(time.time())), 100)
        
        self.noiseless_source = copy.deepcopy(self.source)
        
        if scenario == 1:
            self.avg_signal = np.abs(self.source_1.s_nu)
        elif scenario == 2:
            self.avg_signal = np.abs(self.source_2.s_nu)
        else:
            self.avg_signal = (np.abs(self.source_1.s_nu) + np.abs(self.source_2.s_nu))/2.0
        
        if noise_frac:
            self.source.apply_noise(self.avg_signal*self.noise_frac)
                
        if use_gridding:
            gridding = Gridding(self.source)
            gridding_noiseless = Gridding(self.noiseless_source)
            self.source = gridding.run()
            self.noiseless_source = gridding_noiseless.run()
    
    def apply_noise(self):
        if self.noise_frac:
            self.source.apply_noise(self.avg_signal*self.noise_frac, np.random.RandomState(int(time.time())))
            
    def run(self, lambda_tv:float=None, lambda_tsv:float=None):
        self.parameter = Parameter()
        self.parameter.calculate_cellsize(dataset=self.source)
        
        dft = DFT1D(dataset=self.source, parameter=self.parameter)
        
        self.F_dirty = dft.backward(self.source.data)
        
        idx_noise = np.where(np.abs(self.parameter.phi)>self.parameter.max_faraday_depth/1.5)
        
        self.noise = 0.5*(np.std(self.F_dirty[idx_noise].real) + np.std(self.F_dirty[idx_noise].imag))
        
        if self.use_wavelet:
            #self.wavelet = DiscreteWavelet(wavelet_name=self.use_wavelet, mode="periodization")
            self.wavelet = UndecimatedWavelet(wavelet_name=self.use_wavelet)
        
        self.lambda_l1 = np.sqrt(2.0 * len(self.source.data) + 4.0 * np.sqrt(len(self.source.data)))*self.noise*np.sqrt(0.5)
        #self.lambda_l1 = np.sqrt(len(self.source.data))*self.noise
        #lambda_l1 =np.sqrt(len(self.source.data) + np.sqrt(len(self.source.data)))*self.noise
        
        if lambda_tv is None:
            lambda_tv = 0.0
        
        if lambda_tsv is None:
            lambda_tsv = 0.0
            
        if self.ftransform == "nufft":
            nufft = NUFFT1D(dataset=self.source, parameter=self.parameter, solve=True)
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=nufft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=nufft)
        else:
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=dft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=dft)
            
        l1 = L1(reg=self.lambda_l1)
        tsv = TSV(reg=lambda_tsv)
        tv = TV(reg=lambda_tv)
        F_func = [chi2, l1, tsv]
        f_func = [chi2]
        g_func = [l1, tsv]

        F_obj = OFunction(F_func)
        f_obj = OFunction(f_func)
        g_obj = OFunction(g_func)
        
        self.parameter.data = self.F_dirty
        
        self.parameter.complex_data_to_real()
        
        if self.use_wavelet:
            self.parameter.data = self.wavelet.decompose(self.parameter.data)
        
        opt = FISTA(guess_param=self.parameter, F_obj=F_obj, fx=chi2, gx=g_obj, noise=self.noise, verbose=True)
        self.obj, self.X = opt.run()
        
        if self.use_wavelet is not None:
            self.coeffs = copy.deepcopy(self.X.data)
            k = np.count_nonzero(self.coeffs)
            self.sparsity = k/len(self.coeffs)
            self.gini = gini_coefficient(self.coeffs)
            self.X.data = self.wavelet.reconstruct(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.coeffs)
            
        else:
            k = np.count_nonzero(self.X.data)
            self.sparsity = k/len(self.X.data)
            self.gini = gini_coefficient(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.X.data)
        
        
        self.X.real_data_to_complex()
        
        self.X_residual = dft.backward(self.source.residual)
        
        self.X_restored = self.X.convolve() + self.X_residual
        
        self.res_noise = 0.5*(np.std(self.X_residual.real) + np.std(self.X_residual.imag))
        self.rmse = np.sqrt(np.sum(self.source.residual.real**2 + self.source.residual.imag**2)/(2*len(self.source.residual)))
        meaningful_signal = np.where(np.abs(self.parameter.phi) < self.parameter.max_faraday_depth)
        self.signal = np.mean(np.abs(self.X_restored[meaningful_signal]))
        self.peak_signal = np.max(np.abs(self.X_restored))
        
        self.snr = self.signal / self.res_noise
        self.psnr = self.peak_signal / self.res_noise
        
        print("Signal-to-noise ratio: {0}".format(self.snr))
        print("Peak Signal-to-noise ratio: {0}".format(self.psnr))
        print("Standard deviation: {0}".format(self.res_noise))
        
        self.lags, self.autocorr_res, self.autocorr_res_sq, self.bound, self.percentage_real_in, self.percentage_imag_in, self.percentage_real_in_sq, self.percentage_imag_in_sq = self.source.assess_residuals()
        
        self.residual_comparison = self.noiseless_source.data - self.source.model_data
        
        self.fig, self.ax = plt.subplots(nrows=2, ncols=4, sharey='row', figsize=(18, 5))

        # Data
        self.ax[0,0].plot(self.source.lambda2, self.source.data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,0].plot(self.source.lambda2, self.source.data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,0].plot(self.source.lambda2, np.abs(self.source.data), 'g.', label=r"$|P|$")
        self.ax[0,0].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,0].set_ylabel(r'Jy/beam')
        self.ax[0,0].title.set_text("Data")

        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,0].plot(self.parameter.phi, np.abs(self.F_dirty), 'k-', label=r"|P|")
        self.ax[1,0].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,0].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,0].set_xlim([-1000,1000])

        # Model
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,1].plot(self.source.lambda2, np.abs(self.source.model_data), 'g.', label=r"$|P|$")
        self.ax[0,1].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,1].set_ylabel(r'Jy/beam')
        self.ax[0,1].title.set_text("Model")

        self.ax[1,1].get_shared_y_axes().remove(self.ax[1,1])
        self.ax[1,1].clear()
        self.ax[1,1].plot(self.parameter.phi, self.X.data.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,1].plot(self.parameter.phi, self.X.data.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,1].plot(self.parameter.phi, np.abs(self.X.data), 'k-', label=r"$|P|$")
        self.ax[1,1].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,1].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
        self.ax[1,1].set_xlim([-1000,1000])
        
        # Residual

        self.ax[0,2].plot(self.source.lambda2, self.source.residual.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,2].plot(self.source.lambda2, self.source.residual.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,2].plot(self.source.lambda2, np.abs(self.source.residual), 'g.', label=r"$|P|$")
        self.ax[0,2].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,2].set_ylabel(r'Jy/beam')
        self.ax[0,2].title.set_text("Residual")

        self.ax[1,2].plot(self.parameter.phi, self.X_residual.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,2].plot(self.parameter.phi, self.X_residual.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,2].plot(self.parameter.phi, np.abs(self.X_residual), 'k-', label=r"$|P|$")
        self.ax[1,2].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,2].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,2].set_xlim([-1000,1000])
        
        if self.use_wavelet:
            self.ax[0,3].get_shared_y_axes().remove(self.ax[0,3])
            self.ax[0,3].clear()
            self.ax[0,3].plot(self.coeffs)
            self.ax[0,3].title.set_text("Coefficients")

        self.ax[1,3].plot(self.parameter.phi, self.X_restored.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,3].plot(self.parameter.phi, self.X_restored.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,3].plot(self.parameter.phi, np.abs(self.X_restored), 'k-', label=r"$|P|$")
        self.ax[1,3].set_xlim([-1000,1000])
        self.ax[1,3].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,3].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,3].title.set_text("Restored")
        
        self.fig.tight_layout()

In [8]:
def run_test(source_1, source_2, nsigma, remove_frac, nu_min=1.008e9, nu_max=2.031e9, nchannels=1000, scenario=1, use_wavelet=None):
    nosigma_objs = []
    for remv_frac in remove_frac:
        nosigma_objs.append(Test(nu_min=nu_min, nu_max=nu_max, nchannels=nchannels, noise_frac=0.0, remove_frac=remv_frac, scenario=scenario, source_1=source_1, source_2=source_2, use_wavelet=use_wavelet))

    test_objs = []
    for nsig in nsigma:
        for i in range(0, len(remove_frac)):
            copy_object = copy.deepcopy(nosigma_objs[i])
            copy_object.noise_frac = nsig
            copy_object.apply_noise()
            test_objs.append(copy_object)

    del nosigma_objs

    nid = len(nsigma)*len(remove_frac)
    for _id in range(0, nid):
        test_objs[_id].run()
    
    return test_objs 

In [9]:
# JVLA 1.008 - 2.031 GHz 546 channels
# MeerKAT 0.9 GHz-1.420 GHz 546
# eMERLIN 1.230 - 1.740 GHz 4096
def run_tests(source_1, source_2, nsigma, remove_frac, nsamples, nu_min=1.008e9, nu_max=2.031e9, nchannels=1000, scenario=1, use_wavelet=None):
    m = len(nsigma)
    n = len(remove_frac)
    psnrs = statistics(m, n, nsamples)
    rmses = statistics(m, n, nsamples)
    #noises = statistics(m, n, nsamples)
    sparsities = statistics(m, n, nsamples)
    #ginies = statistics(m, n, nsamples)
    aics = statistics(m, n, nsamples)
    bics = statistics(m, n, nsamples)
    for i in range(0, nsamples):
        test = run_test(source_1, source_2, nsigma, remove_frac, nu_min=nu_min, nu_max=nu_max, nchannels=nchannels, scenario=scenario, use_wavelet=use_wavelet)
        psnrs.cumul(list_to2darray([x.psnr for x in test], n, dtype=np.float32))
        #noises.cumul(list_to2darray([x.res_noise for x in test], n, dtype=np.float32))
        sparsities.cumul(list_to2darray([x.sparsity*100.0 for x in test], n, dtype=np.float32))
        #ginies.cumul(list_to2darray([x.gini for x in test], n, dtype=np.float32))
        rmses.cumul(list_to2darray([x.rmse for x in test], n, dtype=np.float32))
        aics.cumul(list_to2darray([x.aic for x in test], n, dtype=np.float32))
        bics.cumul(list_to2darray([x.bic for x in test], n, dtype=np.float32))
        for t in test:
            del t
        test = []
    
    psnr_mean, psnr_std = psnrs.mean(), psnrs.std()
    rmse_mean, rmse_std = rmses.mean(), rmses.std()
    #noise_mean, noise_std = noises.mean(), noises.std()
    sparsity_mean, sparsity_std = sparsities.mean(), sparsities.std()
    #gini_mean, gini_std = ginies.mean(), ginies.std()
    aic_mean, aic_std = aics.mean(), aics.std()
    bic_mean, bic_std = bics.mean(), bics.std()
    return psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std

In [10]:
source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-200, spectral_idx=1.0)
source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=140, phi_center=200, spectral_idx=1.0)

In [11]:
nsigma = [0.1]
remove_frac = [0.2]
scenarios=[1,2,3]
#use_wavelet= pywt.wavelist(kind="discrete").remove("db1")
#use_wavelet = ["db1", "coif1", "coif2"]
#idx_coif = use_wavelet.index("coif1") - 0.5
use_wavelet = pywt.wavelist(kind="discrete")
use_wavelet.remove("haar")
nwavelets = len(use_wavelet)
idx_coif = [use_wavelet.index("coif1") - 0.5, use_wavelet.index("db1") - 0.5, use_wavelet.index("dmey") - 0.5,  use_wavelet.index("rbio1.1") - 0.5, use_wavelet.index("sym2") - 0.5]
#use_wavelet=None
samples=5

In [12]:
print(use_wavelet)

['bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15', 'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34', 'db35', 'db36', 'db37', 'db38', 'dmey', 'rbio1.1', 'rbio1.3', 'rbio1.5', 'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1', 'rbio3.3', 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5', 'rbio6.8', 'sym2', 'sym3', 'sym4', 'sym5', 'sym6', 'sym7', 'sym8', 'sym9', 'sym10', 'sym11', 'sym12', 'sym13', 'sym14', 'sym15', 'sym16', 'sym17', 'sym18', 'sym19', 'sym20']


In [13]:
# This cell will save the numpy arrays
names = ["PSNR", "RMSE", "AIC", "BIC"] 
scenario_means = np.empty((len(scenarios), len(names), nwavelets), dtype=np.float32)
scenario_stds = np.empty((len(scenarios), len(names), nwavelets), dtype=np.float32)

for i in range(len(scenarios)):
    for j in range(nwavelets):
        psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(source_1, source_2, nsigma, remove_frac, samples, scenario=scenarios[i], use_wavelet=use_wavelet[j])
        scenario_means[i,0,j] = psnr_mean
        scenario_means[i,1,j] = rmse_mean
        scenario_means[i,2,j] = aic_mean
        scenario_means[i,3,j] = bic_mean
        
        scenario_stds[i,0,j] = psnr_std
        scenario_stds[i,1,j] = rmse_std
        scenario_stds[i,2,j] = aic_std
        scenario_stds[i,3,j] = bic_std

FWHM of the main peak of the RMTF: 52.950 rad/m^2
Maximum recovered width structure: 144.188 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 21153.690
Iterations set to 29
Iteration:  0  objective function value: 0.00387
Signal-to-noise ratio: 1.5231031791969705
Peak Signal-to-noise ratio: 196.85340445163914
Standard deviation: 1.501214865129441e-05
FWHM of the main peak of the RMTF: 53.848 rad/m^2
Maximum recovered width structure: 143.897 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 21512.157
Iterations set to 29
Iteration:  0  objective function value: 0.00389
Signal-to-noise ratio: 1.600833486723739
Peak Signal-to-noise ratio: 184.27131445854508
Standard deviation: 1.611990956007503e-05
FWHM of the main peak of the RMTF: 61.480 rad/m^2
Maximum recovered width structure: 144.188 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 24561.323
Iterations set to 29
Iteration:  0  objective function value: 0.0039

/tmp/ipykernel_148453/617857192.py:155: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.fig, self.ax = plt.subplots(nrows=2, ncols=4, sharey='row', figsize=(18, 5))


FWHM of the main peak of the RMTF: 62.649 rad/m^2
Maximum recovered width structure: 142.159 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 25028.436
Iterations set to 29
Iteration:  0  objective function value: 0.00312
Signal-to-noise ratio: 1.5627365865298601
Peak Signal-to-noise ratio: 210.14134597594966
Standard deviation: 1.4384841961145867e-05
FWHM of the main peak of the RMTF: 62.135 rad/m^2
Maximum recovered width structure: 144.188 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 24822.747
Iterations set to 29
Iteration:  0  objective function value: 0.00305
Signal-to-noise ratio: 1.5905910012922
Peak Signal-to-noise ratio: 217.06297577195215
Standard deviation: 1.3998784197610803e-05
FWHM of the main peak of the RMTF: 59.397 rad/m^2
Maximum recovered width structure: 144.188 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 23729.012
Iterations set to 29
Iteration:  0  objective function value: 0.0021

limit_output extension: Maximum message size of 10000 exceeded with 10051 characters

In [14]:
np.save("undecimated_wavelet_jvla_means.npy", scenario_means)
np.save("undecimated_wavelet_jvla_stds.npy", scenario_stds)

In [15]:
"""
_id = 0 
fig, ax = plt.subplots(nrows=len(scenarios), ncols=len(names), sharey='none', sharex='all', figsize=(18, 5))
#cmap = plt.get_cmap('tab20')
cmap = plt.get_cmap('plasma')
colors = [cmap(i) for i in np.linspace(0, 1, nwavelets)]
scenarios_means = []
scenarios_stds = []
for sc in scenarios:
    _id=0
    for wav in use_wavelet:
        psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(source_1, source_2, nsigma, remove_frac, samples, scenario=sc, use_wavelet=wav)
        means = [psnr_mean, rmse_mean, aic_mean, bic_mean]
        stds = [psnr_std, rmse_std, aic_std, bic_std]
        scenarios_means.append(means)
        scenarios_stds.append(stds)
        for f in range(0, len(names)):
            ax[sc-1,f].plot(_id, means[f][0,0], label=wav, color=colors[_id])
            ax[sc-1,f].errorbar(_id, means[f][0,0], yerr = stds[f][0,0], fmt ='.', capsize=4, color=colors[_id])
            #if names[f] == "PSNR" and sc==3:
                #ax[sc-1,f].legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), ncol=19, fancybox=True, shadow=True, title="Wavelets")
            ax[sc-1,f].set_ylabel(names[f])
            if sc==3:
                ax[sc-1,f].set_xlabel("Wavelet")

        _id += 1

for k in range(len(idx_coif)):
    for i in range(len(scenarios)):
        for j in range(len(names)):
            ax[i,j].axvline(x = idx_coif[k], color = 'k', linestyle="--", alpha=0.8)
            
for i in range(len(scenarios)):
    ax[i,2].set_ylim([-25000,-20000])
    ax[i,3].set_ylim([-25000,-20000])

    #ax[i,2].set_ylim([-25000,-15000]) undecimated
    #ax[i,3].set_ylim([-25000,17500]) undecimated

ofile_name = "wavelets.png"
#fig.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=19, fancybox=True, shadow=True, title="Wavelets")
#ax[len(scenarios)-1, 1].legend(loc='upper left', bbox_to_anchor=[1.0, -0.5], ncol=19, fancybox=True, shadow=True, title="Wavelets")
h, l = ax[0,0].get_legend_handles_labels()
legend = fig.legend(h, l, loc='lower center', ncol=18, fancybox=True, bbox_to_anchor=[0.52, -0.375], shadow=True, title="Wavelets")
fig.tight_layout()
if os.path.isfile(ofile_name):
    os.remove(ofile_name)   # Opt.: os.system("rm "+strFile)
fig.savefig(ofile_name, dpi=100, bbox_inches='tight')#
"""

'\n_id = 0 \nfig, ax = plt.subplots(nrows=len(scenarios), ncols=len(names), sharey=\'none\', sharex=\'all\', figsize=(18, 5))\n#cmap = plt.get_cmap(\'tab20\')\ncmap = plt.get_cmap(\'plasma\')\ncolors = [cmap(i) for i in np.linspace(0, 1, nwavelets)]\nscenarios_means = []\nscenarios_stds = []\nfor sc in scenarios:\n    _id=0\n    for wav in use_wavelet:\n        psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(source_1, source_2, nsigma, remove_frac, samples, scenario=sc, use_wavelet=wav)\n        means = [psnr_mean, rmse_mean, aic_mean, bic_mean]\n        stds = [psnr_std, rmse_std, aic_std, bic_std]\n        scenarios_means.append(means)\n        scenarios_stds.append(stds)\n        for f in range(0, len(names)):\n            ax[sc-1,f].plot(_id, means[f][0,0], label=wav, color=colors[_id])\n            ax[sc-1,f].errorbar(_id, means[f][0,0], yerr = stds[f][0,0], fmt =\'.\', capsize=4, color=colors[_id])\n         